#### Scala can already make combinations of set elements

In [1]:
val things = Set("phone","card","key","vase","mirror","water","trail mix", "kale")

In [2]:
things

Set(trail mix, kale, key, water, card, mirror, vase, phone)

In [3]:
things.subsets(3) foreach println

Set(trail mix, kale, key)
Set(trail mix, kale, water)
Set(trail mix, kale, card)
Set(trail mix, kale, mirror)
Set(trail mix, kale, vase)
Set(trail mix, kale, phone)
Set(trail mix, key, water)
Set(trail mix, key, card)
Set(trail mix, key, mirror)
Set(trail mix, key, vase)
Set(trail mix, key, phone)
Set(trail mix, water, card)
Set(trail mix, water, mirror)
Set(trail mix, water, vase)
Set(trail mix, water, phone)
Set(trail mix, card, mirror)
Set(trail mix, card, vase)
Set(trail mix, card, phone)
Set(trail mix, mirror, vase)
Set(trail mix, mirror, phone)
Set(trail mix, vase, phone)
Set(kale, key, water)
Set(kale, key, card)
Set(kale, key, mirror)
Set(kale, key, vase)
Set(kale, key, phone)
Set(kale, water, card)
Set(kale, water, mirror)
Set(kale, water, vase)
Set(kale, water, phone)
Set(kale, card, mirror)
Set(kale, card, vase)
Set(kale, card, phone)
Set(kale, mirror, vase)
Set(kale, mirror, phone)
Set(kale, vase, phone)
Set(key, water, card)
Set(key, water, mirror)
Set(key, water, vase)
Se

#### Note: Should really model a table with receipt_id, set(skus)  or date, receipt_id, skus, primary key ((date,store),receiptid) just to keep partitions balanced.   This solution just makes an rdd like that with the existing tables using group by, but change it out to and rdd based on the new table 

In [4]:
val items = sc.cassandraTable("retail","receipts").select("receipt_id", "product_id").map( r => (r.getString(0), r.getString(1)))

In [5]:
val itemsByReceipt = items.groupByKey.sample(false, 0.1)

In [6]:
itemsByReceipt.cache

PartitionwiseSampledRDD[5] at sample at <console>:19

for each group, pull out the set, and subsets(3) will give you an **iterator** over all of the combos of 3.  We need to list to materialize the subsets.  We then flatmap it to yield an RDD of sets of 3 items from the same receipt.

In [7]:
val combinations = itemsByReceipt.flatMap(_._2.toSet.subsets(3).toList)
combinations.cache
combinations take 10

Array(Set(9000479, K8E30AA, HA124A1#58E), Set(9000479, K8E30AA, HDMIMM20HS), Set(9000479, K8E30AA, U2QV5E), Set(9000479, K8E30AA, 400-00), Set(9000479, K8E30AA, 20002584), Set(9000479, K8E30AA, MUSCP0320), Set(9000479, HA124A1#58E, HDMIMM20HS), Set(9000479, HA124A1#58E, U2QV5E), Set(9000479, HA124A1#58E, 400-00), Set(9000479, HA124A1#58E, 20002584))

count the prevalence in an rdd.

In [14]:
combinations.map((_,1)).reduceByKey(_+_).sortBy(-_._2) take 10 foreach println

(Set(WG-C20NB, AG647A, UE40H6670SZXZT),6)
(Set(1KH0.002.01, 0048, HA103A3#7GF),6)
(Set(KTL-TP3C/8G, CB458A, DP2VGA),6)
(Set(UE46H6203AWXXN, 32PF7320, KTN-PM667/1G),6)
(Set(T110-8319, 40K1042, A0J99A),6)
(Set(CB458A, U4544E, DP2VGA),6)
(Set(405-31, HA103A3#7GD, U3VK1E),6)
(Set(U4CQ1PE, HA103A3#87F, 7020-6607),6)
(Set(HA103A3#7GD, U3VK1E, 42LM340S),5)
(Set(461137-B21, DCS-3411, DSS-101-BLACK),5)


### All Together
* Note: Items are just (receipt, product_id) pairs
```
    val items = sc.cassandraTable("retail","receipts").select("receipt_id", "product_id").map( r => (r.getString(0), r.getString(1)))
```
* To save time, we look at a sample of 10% of the receipts

In [23]:
items.groupByKey.sample(false, 0.1).flatMap(_._2.toSet.subsets(3).toList).map((_,1)).reduceByKey(_+_).sortBy(-_._2) take 10 foreach println

(Set(U9P73E, BWM685EA1, HQ284E),7)
(Set(7020-7611, 406551710, 3020-3769),7)
(Set(DVM63HDV, 508-46, U3SN0E),6)
(Set(3610-3554, 40X7540, UX151E),6)
(Set(UE40H5570SSXZG, 402-57, A0J99A),6)
(Set(90-XB1X00MU00020, 405-48, H2W15AA#ABB),6)
(Set(7100V_DNM, HA103A3#7GD, 42LM340S),5)
(Set(DMC-1002, MUSKI0172, U3WB3E),5)
(Set(I16988, GE60 2PE-494TR, CA026D7020MT11),5)
(Set(T110-2303, U4VD6E, U3RS0PE),5)


## And all of the Data

In [24]:
items.groupByKey.flatMap(_._2.toSet.subsets(3).toList).map((_,1)).reduceByKey(_+_).sortBy(-_._2) take 10 foreach println

(Set(25204900, 379614810, 460-10),22)
(Set(U3VF7E, 32L1400UC, ECCLSPRMULT),21)
(Set(UE40H6620SVXZG, 90R-N00WR1000T, 133-28),20)
(Set(TK-410, 71P9158, TQXS48),19)
(Set(KDL-40HX855, P566-016, 3020-8823),19)
(Set(1T02G10EU0, 437372-B21, ILCE7M2KB.CEC),18)
(Set(3600531082703, UE48H6410SSXXH, U3SP7E),18)
(Set(PS23N1THIN50, U3WL1E, UE55F6770SSXXN),18)
(Set(HA454A1#000, U6PJ6E, RF5-1835-000CN),18)
(Set(PSCLUU-02G075, 420-07, U4HE8E),18)


In [25]:
var results = items.groupByKey.sample(false, 0.1).flatMap(_._2.toSet.subsets(3).toList).map((_,1)).reduceByKey(_+_).sortBy(-_._2) ta

In [32]:
results.map(_._1.toString).foreach(println)

Set(DIR-615, HQ-HP10IPB, A3L980B02M-REDS)
Set(GF343AA, CZ155A, P-HD-2PXB57O)
Set(KVR1333D3E9SK3/24G, 3600522891437, MUSCP0812)
Set(HA103A3#7GQ, U9P78E, 5810-0177)
Set(US765PE, EL800USBIEC, U9Q44E)
Set(SDRW-08D3S-U, MW-140BTF, 73P2656)
Set(GT-S5830OKI, 8287B006, 9020-3905)
Set(U4GM0PE, UE32H6470SSXZG, U2YQ5PE)
Set(440-84, U3SQ2PE, E2Q81AA)
Set(NWZE574B, BX80621E52630, 6M.GCGD5.001)


In [44]:
kernel.magics.html("<table>" + results.map(x => "<tr><td>" + x._1.mkString(",") + " </td></tr>").mkString(" ") + "</table>")

"DIR-615,HQ-HP10IPB,A3L980B02M-REDS"
"GF343AA,CZ155A,P-HD-2PXB57O"
"KVR1333D3E9SK3/24G,3600522891437,MUSCP0812"
"HA103A3#7GQ,U9P78E,5810-0177"
"US765PE,EL800USBIEC,U9Q44E"
"SDRW-08D3S-U,MW-140BTF,73P2656"
"GT-S5830OKI,8287B006,9020-3905"
"U4GM0PE,UE32H6470SSXZG,U2YQ5PE"
"440-84,U3SQ2PE,E2Q81AA"
"NWZE574B,BX80621E52630,6M.GCGD5.001"


In [61]:
val productMap = sc.cassandraTable("retail","products_by_id").select("product_id","title").as((_:String,_:String)).collect.toList.toMap

In [64]:
val names = results.map(_._1.map(x => (x,productMap{x})))

In [66]:
kernel.magics.html("<table>" + names.map(x => "<tr><td>" + x.mkString(",") + " </td></tr>").mkString(" ") + "</table>")

"(DIR-615,D-Link DIR-615 Wireless N Router),(HQ-HP10IPB,HQ HP10IPB),(A3L980B02M-REDS,Belkin SNAGLESS CAT6 PATCH CABLE)"
"(GF343AA,HP 16X SATA SuperMulti LightScribe Drive),(CZ155A,HP Officejet 6600 e-AiO),(P-HD-2PXB57O,Samsung P-HD-2PXB57O)"
"(KVR1333D3E9SK3/24G,Kingston Technology ValueRAM KVR1333D3E9SK3),(3600522891437,L’Oréal Paris Color Riche Accords Intenses 703 Oud Obsession),(MUSCP0812,Muvit MUSCP0812 screen protector)"
"(HA103A3#7GQ,HP 3y 4H 13x5 MSA1500 HW Support),(U9P78E,HP 4y6h24x7w/CDMR5830-96G Switch HW Suppo),(5810-0177,DELL Precision T5810)"
"(US765PE,HP 1y PW 4h 13x5 1606 Base Swtch HW Supp),(EL800USBIEC,Eaton Ellipse ECO 800 USB IEC),(U9Q44E,HP 4y6h24x7w/CDMR25/26xx/Wireless HWSupp)"
"(SDRW-08D3S-U,ASUS SDRW-08D3S-U),(MW-140BTF,Brother MW-140BT Mobile Printer),(73P2656,Lenovo 73P2656)"
"(GT-S5830OKI,Samsung Galaxy Ace GT-S5830 0.158GB Black),(8287B006,Canon PG-545/CL-546 Multipack),(9020-3905,DELL OptiPlex 9020)"
"(U4GM0PE,HP 1y PWNbdw/CDMR95/75xxldblnc mod FCSVC),(UE32H6470SSXZG,Samsung UE32H6470),(U2YQ5PE,HP 1y PW Nbd w/DMR DL785 G6 FC SVC)"
"(440-84,Sandberg MicroUSB Cable Flat 0.15m),(U3SQ2PE,HP 1y PW Nbd Exch MSR900 Rtr FC SVC),(E2Q81AA,HP Z820 Xeon E5-2667v2 3.3GHz 1866MHz 8 Core 2nd CPU)"
"(NWZE574B,Sony NWZ-E574 8GB Walkman® Video MP3 player),(BX80621E52630,Intel Xeon E5-2630),(6M.GCGD5.001,Acer 6M.GCGD5.001)"


In [8]:
%%html <b> something bold </b>

something bold